In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 문제
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 17,8

import warnings
warnings.filterwarnings(action='ignore')

# **대구 교통 사고 피해 예측 AI 경진대회 Baseline Code**

## **Fixed Random Seed**  

seed 값에 의해 동일한 코드를 사용해도 결과가 다를 수 있기에, 동일한 결과를 위해 seed 값을 고정시킵니다

In [2]:
import os
import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

## **데이터 불러오기 및 상위행 확인**  

train.csv, test.csv 파일을 로드하여 상위행을 출력해 봅시다

In [3]:
import pandas as pd
from datetime import datetime

train = pd.read_csv('./open/train.csv', encoding = "utf-8", low_memory=False)
test = pd.read_csv('./open/test.csv', encoding = "utf-8", low_memory=False)

display(train.head())
display(test.tail())

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
10958,ACCIDENT_50567,2022-12-31 18,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차
10959,ACCIDENT_50568,2022-12-31 18,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차
10960,ACCIDENT_50569,2022-12-31 20,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차
10961,ACCIDENT_50570,2022-12-31 20,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차
10962,ACCIDENT_50571,2022-12-31 21,토요일,맑음,대구광역시 동구 효목동,교차로 - 교차로부근,건조,차대차


## **train, test 데이터 기간 확인하기**  

학습(train) 데이터의 기간과 예측 대상이 되는 test 데이터의 기간을 살펴 봅니다

In [4]:
display(f"train : {train.iloc[0]['사고일시']} ~ {train.iloc[-1]['사고일시']}")
display(f"test : {test.iloc[0]['사고일시']} ~ {test.iloc[-1]['사고일시']}")

'train : 2019-01-01 00 ~ 2021-12-31 23'

'test : 2022-01-01 01 ~ 2022-12-31 21'

### Data Info

In [5]:
#'사고일시' 데이터 타입을 object -> datetime으로  바꾸기
train['사고일시'] = pd.to_datetime(train['사고일시'])
test['사고일시'] = pd.to_datetime(test['사고일시'])

### 날짜 데이터 파싱하기

In [6]:
train['year'] = train.사고일시.dt.year
train['month'] = train.사고일시.dt.month
train['day'] = train.사고일시.dt.day
train['hour'] = train.사고일시.dt.hour

In [7]:
test['year'] = test.사고일시.dt.year
test['month'] = test.사고일시.dt.month
test['day'] = test.사고일시.dt.day
test['hour'] = test.사고일시.dt.hour

## **데이터 전처리**  

현재 '사고일시', '시군구', '도로형태' 컬럼은 반복되는 패턴으로 여러 정보를 포함하고 있습니다
이런 반복되는 패턴을 일반화하면 pandas에서 제공하는 str.extract를 통해 한 번에 추출 가능합니다  

## **파생 변수 생성 1 : 날짜, 시간정보 생성**

'사고일시' 컬럼으로 부터 연도, 월, 일, 시간 정보 추출 및 변환 합니다

In [8]:
import pandas as pd
from datetime import datetime

train_org = pd.read_csv('./open/train.csv',encoding = "utf-8", low_memory=False)
test_org = pd.read_csv('./open/test.csv', encoding = "utf-8", low_memory=False)

display(train_org.head())
display(test_org.tail())

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
10958,ACCIDENT_50567,2022-12-31 18,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차
10959,ACCIDENT_50568,2022-12-31 18,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차
10960,ACCIDENT_50569,2022-12-31 20,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차
10961,ACCIDENT_50570,2022-12-31 20,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차
10962,ACCIDENT_50571,2022-12-31 21,토요일,맑음,대구광역시 동구 효목동,교차로 - 교차로부근,건조,차대차


In [9]:
train_df = train_org.copy()
test_df = test_org.copy()

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

train_df[['연', '월', '일', '시간']] = train_org['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다
train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다

# 해당 과정을 test_x에 대해서도 반복해줍니다
test_df[['연', '월', '일', '시간']] = test_org['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류',\n       '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간'], dtype='object')"

## **파생 변수 생성 2 : 공간(위치) 정보 생성**

'시군구' 컬럼으로부터 의미 있는 공산 정보를 추출 및 변환 합니다

In [10]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_org['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test_org['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반',\n       '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간', '도시', '구', '동'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시',\n       '구', '동'],\n      dtype='object')"

## **파생 변수 추출 3 : 도로 형태 정보 추출**  

'도로형태' 컬럼은 '단일로 - 기타'와 같은 패턴으로 구성되어 있습니다. 이를 두종류의 독립된 정보로 보고 두개의 컬럼으로 분리하여 생성합니다.

In [11]:
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_org['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

test_df[['도로형태1', '도로형태2']] = test_org['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반', '가해운전자 차종',\n       '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별',\n       '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수', 'ECLO', '연',\n       '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '동',\n       '도로형태1', '도로형태2'],\n      dtype='object')"

## train, test 컬럼 수 맞추기

test에 있는 컬럼들만 사용하여 train 데이터를 구성했습니다.

In [12]:
train_df= train_df[['ID','요일','기상상태','노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구','동', '도로형태1', '도로형태2']]
train_df

,ID,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_00000,화요일,맑음,건조,차대사람,2019,1,1,0,대구광역시,중구,대신동,단일로,기타
1,ACCIDENT_00001,화요일,흐림,건조,차대사람,2019,1,1,0,대구광역시,달서구,감삼동,단일로,기타
2,ACCIDENT_00002,화요일,맑음,건조,차대사람,2019,1,1,1,대구광역시,수성구,두산동,단일로,기타
3,ACCIDENT_00003,화요일,맑음,건조,차대차,2019,1,1,2,대구광역시,북구,복현동,단일로,기타
4,ACCIDENT_00004,화요일,맑음,건조,차대차,2019,1,1,4,대구광역시,동구,신암동,단일로,기타
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39604,금요일,맑음,건조,차대차,2021,12,31,19,대구광역시,수성구,수성동3가,교차로,교차로안
39605,ACCIDENT_39605,금요일,맑음,건조,차대차,2021,12,31,19,대구광역시,달서구,상인동,단일로,기타
39606,ACCIDENT_39606,금요일,맑음,건조,차대차,2021,12,31,21,대구광역시,달서구,월성동,교차로,교차로안
39607,ACCIDENT_39607,금요일,맑음,건조,차대차,2021,12,31,22,대구광역시,달서구,장동,기타,기타


## **전처리 결과 확인**

지금까지 전처리한 결과를 확인해 봅시다

In [13]:
display(train_df.head())
display(test_df.head())

,ID,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_00000,화요일,맑음,건조,차대사람,2019,1,1,0,대구광역시,중구,대신동,단일로,기타
1,ACCIDENT_00001,화요일,흐림,건조,차대사람,2019,1,1,0,대구광역시,달서구,감삼동,단일로,기타
2,ACCIDENT_00002,화요일,맑음,건조,차대사람,2019,1,1,1,대구광역시,수성구,두산동,단일로,기타
3,ACCIDENT_00003,화요일,맑음,건조,차대차,2019,1,1,2,대구광역시,북구,복현동,단일로,기타
4,ACCIDENT_00004,화요일,맑음,건조,차대차,2019,1,1,4,대구광역시,동구,신암동,단일로,기타


,ID,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_39609,토요일,맑음,건조,차대사람,2022,1,1,1,대구광역시,수성구,상동,교차로,교차로안
1,ACCIDENT_39610,토요일,맑음,건조,차대사람,2022,1,1,1,대구광역시,수성구,지산동,단일로,기타
2,ACCIDENT_39611,토요일,맑음,건조,차대차,2022,1,1,4,대구광역시,수성구,수성동2가,교차로,교차로안
3,ACCIDENT_39612,토요일,맑음,건조,차대차,2022,1,1,4,대구광역시,수성구,신매동,단일로,기타
4,ACCIDENT_39613,토요일,맑음,건조,차대차,2022,1,1,6,대구광역시,달서구,감삼동,교차로,교차로안


In [16]:
display(train_df.info())
display(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      39609 non-null  object
 1   요일      39609 non-null  object
 2   기상상태    39609 non-null  object
 3   노면상태    39609 non-null  object
 4   사고유형    39609 non-null  object
 5   연       39609 non-null  int64 
 6   월       39609 non-null  int64 
 7   일       39609 non-null  int64 
 8   시간      39609 non-null  int64 
 9   도시      39609 non-null  object
 10  구       39609 non-null  object
 11  동       39609 non-null  object
 12  도로형태1   39609 non-null  object
 13  도로형태2   39609 non-null  object
dtypes: int64(4), object(10)
memory usage: 4.2+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10963 non-null  object
 1   요일      10963 non-null  object
 2   기상상태    10963 non-null  object
 3   노면상태    10963 non-null  object
 4   사고유형    10963 non-null  object
 5   연       10963 non-null  int64 
 6   월       10963 non-null  int64 
 7   일       10963 non-null  int64 
 8   시간      10963 non-null  int64 
 9   도시      10963 non-null  object
 10  구       10963 non-null  object
 11  동       10963 non-null  object
 12  도로형태1   10963 non-null  object
 13  도로형태2   10963 non-null  object
dtypes: int64(4), object(10)
memory usage: 1.2+ MB


None

## 전처리를 위해 train과 test 데이터 합치기

In [17]:
# 전처리를 위해 train과 test 데이터 합치기
data = pd.concat([train_df, test_df], sort=False)

# **------------------------------------------------------**

## ✔️Encoding - 파일 추가 필요!

# **------------------------------------------------------**

### 통합된 데이터 세트(data)를 다시 train과 test 세트로 분리
- 합쳐둔 data 날짜에 맞춰서 다시 train_new, test_new로 분리하기

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  object
 2   기상상태    50572 non-null  object
 3   노면상태    50572 non-null  object
 4   사고유형    50572 non-null  object
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  object
 11  동       50572 non-null  object
 12  도로형태1   50572 non-null  object
 13  도로형태2   50572 non-null  object
dtypes: int64(4), object(10)
memory usage: 5.8+ MB


In [19]:
train_new = data[data['연'].isin([2019, 2020, 2021])]
test_new = data[data['연'] == 2022]

# train_new  = data[:len(train)]
# test_new = data[len(train):]
# 이렇게 하는 방법도 있음

In [22]:
train_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39609 entries, 0 to 39608
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      39609 non-null  object
 1   요일      39609 non-null  object
 2   기상상태    39609 non-null  object
 3   노면상태    39609 non-null  object
 4   사고유형    39609 non-null  object
 5   연       39609 non-null  int64 
 6   월       39609 non-null  int64 
 7   일       39609 non-null  int64 
 8   시간      39609 non-null  int64 
 9   도시      39609 non-null  object
 10  구       39609 non-null  object
 11  동       39609 non-null  object
 12  도로형태1   39609 non-null  object
 13  도로형태2   39609 non-null  object
dtypes: int64(4), object(10)
memory usage: 4.5+ MB


In [23]:
test_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10963 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10963 non-null  object
 1   요일      10963 non-null  object
 2   기상상태    10963 non-null  object
 3   노면상태    10963 non-null  object
 4   사고유형    10963 non-null  object
 5   연       10963 non-null  int64 
 6   월       10963 non-null  int64 
 7   일       10963 non-null  int64 
 8   시간      10963 non-null  int64 
 9   도시      10963 non-null  object
 10  구       10963 non-null  object
 11  동       10963 non-null  object
 12  도로형태1   10963 non-null  object
 13  도로형태2   10963 non-null  object
dtypes: int64(4), object(10)
memory usage: 1.3+ MB


### 학습시킬 feature 선택하기 

In [20]:
train_new.columns

Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '동',
       '도로형태1', '도로형태2'],
      dtype='object')

In [21]:
test_new.columns

Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '동',
       '도로형태1', '도로형태2'],
      dtype='object')

In [ ]:
feature =['요일','기상상태','노면상태','사고유형','연','월','시간',,,,,,] 
#예측에 넣을 feature추가하기

### (참고) train, test 데이터의 독립변수, 종속 변수 분리
- [예측에 모든 컬럼 사용하는 법, 내가 고른 컬럼만 사용하는 법] 중에 원하는 방법으로 사용하기!!

In [ ]:
# 예측에 모든 컬럼 사용하는 법.
X_test = test_new.drop(columns=['ID']).copy()
X_train = train_new[X_test.columns].copy()

In [ ]:
# 예측에 내가 고른 컬럼만 사용하는 법.
X_train = train_new[feature]
X_test = test_new[feature]

In [ ]:
y_train = train['ECLO'].copy()

### (참고) 데이터 분할

- 주어진 데이터셋이 이미 훈련용(train)과 테스트용(test)으로 나뉘어져 있으므로 train_test_split 함수를 사용하여 데이터를 분할할 필요 없음!
- 그렇지만, 훈련데이터를 더 작은 훈련 세트(train set)와 검증 세트(validation set)로 나누는 것은 필요함.
> - train data set의 피쳐를 X, 타겟을 y로 분리
> - train data set를 훈련(train)과 검증(valid)로 나눈다.
> - train:valid= 7:3 로 나누어 본다
>> - 훈련 세트(train set): 모델을 훈련하는 데 사용됩니다.
>> - 검증 세트(validation set): 모델을 튜닝하고 성능을 평가하는 데 사용됩니다. 이를 통해 모델이 훈련 데이터에 과적합되지 않고 다른 데이터에 대해서도 일반화될 수 있는지 확인할 수 있습니다.
> - 대회에서는 주어진 테스트 데이터가 숨겨져 있으므로 모델을 평가하려면 모델의 예측을 검증 세트에 대해 평가하고, 최종적으로는 테스트 데이터에 대한 예측을 제출해야 합니다.

In [ ]:
from sklearn.model_selection import train_test_split 
X_test1= train_new.drop('ID', axis=1)
y_test1= train['ECLO']

In [ ]:
#train data set를 train과 validation data set로 나누기
X_train, X_valid, y_train, y_valid = train_test_split(X_test1, y_test1, test_size=0.3, random_state=0)

In [ ]:
# 데이터 사이즈 확인
print("Size of X_train is:{}\nSize of y_train is:{}\nSize of X_valid is:{}\nSize of Y_valid is:{}\n".format(
    X_train.shape,y_train.shape,X_valid.shape,y_valid.shape))

## Model 구현

- Logistic Regression, Decision Tree, Random Forest Classifier, KNN, ......

## Model Train & Prediction

**모델 적용**
- 선택한 모델로 test 결과 저장하기
- 검증 데이터세트의 정확도가 높은 모델을 채택!

In [24]:
# #변수 이름 선택한 모델명으로 설정하기!!!

# from sklearn.tree import DecisionTreeRegressor
# dtr = DecisionTreeRegressor()
# dtr.fit(X_train, y_train)

# prediction_dtr= dtr.predict(X_test)
# prediction_dtr

### - RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 1000, 
                             max_depth = 9, 
                             random_state=37, 
                             max_features =0.8, 
                             n_jobs=-1)

In [ ]:
rfr.fit(X_train, y_train)

prediction_rfr= rfr.predict(X_test)
prediction_rfr

In [ ]:
#random forest는 feature importance 정보 제공 
sorted_idx = rfr.feature_importances_.argsort()
plt.barh(X.columns[sorted_idx], rfr.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

### (참고) 평가함수 -RMSLE

리더보드 제출 전 RMSLE 점수 계산해볼 수 있는 코드입니다. 실제 점수와는 조금 다를수 있어요!

In [ ]:
# from sklearn.metrics import mean_squared_log_error
# # RMSLE 계산
# rmsle = mean_squared_log_error(y_test, y_pred, squared=False)

# # 결과 출력
# print(f'RMSLE: {rmsle}')

In [ ]:
import numpy as np
from sklearn.metrics import make_scorer

def rmsle(predict, actual):
    log_predict= np.log(predict +1)
    log_actual = np.log(actual+1)
    
    difference = log_predict - log_actual
    difference = np.square(difference)
    
    mean_difference = difference.mean()
    
    score = np.sqrt(mean_difference)
    
    return score

rmsle_score = make_scorer(rmsle)

In [ ]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(model , X_train, y_train, cv =5, 
                        scoring=rmsle_score).mean()

print('현재 점수는 {0:.5f}'.format(score))

### Submission

In [ ]:
##형식 복붙해서 사용하시는 모델에 맞춰서 저장해주세요!
# sample_submission = pd.read_csv('open/sample_submission.csv')
# 모델이름_submission = sample_submission.copy()
# 모델이름_submission['ECLO'] = prediction_모델이름
# 모델이름_submission


# 모델이름_submission.to_csv('모델이름_submission.csv')

In [ ]:
sample_submission = pd.read_csv('open/sample_submission.csv')
sample_submission

In [ ]:
# 위코드 해서 안되면 아래꺼 써보기...
# sample_submission = pd.read_csv('open/sample_submission.csv', index_col=0)
# sample_submission

In [ ]:
random_forest_submission = sample_submission.copy()
random_forest_submission['ECLO'] = prediction_rfr
random_forest_submission

In [ ]:
random_forest_submission.to_csv('random_forest_submission.csv')